https://www.kaggle.com/datasets/naiyakhalid/flood-prediction-dataset/data

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [145]:
df = pd.read_csv("flood.csv")
df.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,3,8,6,6,4,4,6,2,3,2,...,10,7,4,2,3,4,3,2,6,0.450
1,8,4,5,7,7,9,1,5,5,4,...,9,2,6,2,1,1,9,1,3,0.475
2,3,10,4,1,7,5,4,7,4,9,...,7,4,4,8,6,1,8,3,6,0.515
3,4,4,2,7,3,4,1,4,6,4,...,4,2,6,6,8,8,6,6,10,0.520
4,3,7,5,2,5,8,5,2,7,5,...,7,6,5,3,3,4,4,3,4,0.475


In [103]:
df.duplicated().sum()

0

In [105]:
df.isnull().sum()

MonsoonIntensity                   0
TopographyDrainage                 0
RiverManagement                    0
Deforestation                      0
Urbanization                       0
ClimateChange                      0
DamsQuality                        0
Siltation                          0
AgriculturalPractices              0
Encroachments                      0
IneffectiveDisasterPreparedness    0
DrainageSystems                    0
CoastalVulnerability               0
Landslides                         0
Watersheds                         0
DeterioratingInfrastructure        0
PopulationScore                    0
WetlandLoss                        0
InadequatePlanning                 0
PoliticalFactors                   0
FloodProbability                   0
dtype: int64

In [147]:
df["FloodProbability"]

0        0.450
1        0.475
2        0.515
3        0.520
4        0.475
         ...  
49995    0.535
49996    0.510
49997    0.430
49998    0.515
49999    0.580
Name: FloodProbability, Length: 50000, dtype: float64

In [155]:
df["FloodProability"] = df["FloodProability"].apply(lambda x: 1 if x > 0.5 else 0)

In [157]:
df["FloodProbability"].value_counts()

FloodProbability
0.500    2018
0.490    1984
0.495    1975
0.485    1960
0.505    1945
         ... 
0.700       1
0.715       1
0.710       1
0.285       1
0.725       1
Name: count, Length: 83, dtype: int64

In [113]:
for col in df.drop("FloodProbability", axis = 1).columns.to_list():
    df[col] = df[col].astype("int8")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   MonsoonIntensity                 50000 non-null  int8   
 1   TopographyDrainage               50000 non-null  int8   
 2   RiverManagement                  50000 non-null  int8   
 3   Deforestation                    50000 non-null  int8   
 4   Urbanization                     50000 non-null  int8   
 5   ClimateChange                    50000 non-null  int8   
 6   DamsQuality                      50000 non-null  int8   
 7   Siltation                        50000 non-null  int8   
 8   AgriculturalPractices            50000 non-null  int8   
 9   Encroachments                    50000 non-null  int8   
 10  IneffectiveDisasterPreparedness  50000 non-null  int8   
 11  DrainageSystems                  50000 non-null  int8   
 12  CoastalVulnerabili

In [115]:
X = df.drop("FloodProbability", axis = 1)
y = df["FloodProbability"]

In [117]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [161]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_jobs = -1)
rfr.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

In [163]:
from sklearn.metrics import r2_score
print(r2_score(y_test, rfr.predict(X_test)))

0.5161648797283616


In [167]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [169]:
r2_score(y_test, lr.predict(X_test))

0.6327702192622013

In [173]:
from sklearn.ensemble import AdaBoostRegressor
abr = AdaBoostRegressor(estimator = LinearRegression(), n_estimators = 100)
abr.fit(X_train, y_train)
r2_score(y_test, abr.predict(X_test))

0.6276336963278515

In [185]:
from sklearn.ensemble import ExtraTreesRegressor, BaggingRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, VotingRegressor
import xgboost as xgb
import catboost as cb
import lightgbm as lgbm
estimators = [
    ("etr", ExtraTreesRegressor(n_estimators = 200)),
    ("br", BaggingRegressor(n_estimators = 100)),
    ("gbr", GradientBoostingRegressor(n_estimators = 100)),
    ("hgbr", HistGradientBoostingRegressor()),
    ("lgbm", lgbm.LGBMRegressor()),
    ("xgbr", xgb.XGBRegressor()),
    ("catboost", cb.CatBoostRegressor())
]

In [187]:
vc = VotingRegressor(estimators = estimators, n_jobs = -1)
vc.fit(X_train, y_train)

VotingRegressor(estimators=[('etr', ExtraTreesRegressor(n_estimators=200)),
                            ('br', BaggingRegressor(n_estimators=100)),
                            ('gbr', GradientBoostingRegressor()),
                            ('hgbr', HistGradientBoostingRegressor()),
                            ('lgbm', LGBMRegressor()),
                            ('xgbr',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,...
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...)),
                            ('catboost',
                             <catboost.core.CatBoostRegressor object at 0x0000020B3E0948F0>)],
                n_jobs=-1)

In [189]:
r2_score(y_test, vc.predict(X_test))

0.6072700440432827